# AGM | Unlocking AGM's Vision: The Role of NoSQL Technologies

#### Notebook Contents
* **(1) General Setup** - Library Imports and Supporting Code and Functions
* **(2) Prepping Data**
* **(3) SQL Analysis**
* **(4) Neo4j Model Setup**

### 1.1 Library Imports

In [1]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

from geographiclib.geodesic import Geodesic

### 1.2 Supporting code and functions for analysis

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [3]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

In [4]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [5]:
session = driver.session(database="neo4j")

In [6]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [7]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [8]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [9]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [10]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [11]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [12]:
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        print("\n--------------------------------")
        print("   Total Cost (Seconds): ", total_cost)
        print("   Minutes: ", round(total_cost / 60.0,1))
        print("--------------------------------")
        
        nodes = r['nodes']
        costs = r['costs']
        
        i = 0
        previous = 0
        
        for n in nodes:
            
            print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))
            
            previous = int(costs[i])
            i += 1
    

In [13]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [14]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    print("\n-------------------------------------------------------------------------------")
    print("  Zip Codes within " + str(miles) + " mile(s) of " + station + " BART Station")
    print("-------------------------------------------------------------------------------\n")
    
    total_population = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        print("     zip:", zip, "  population: ", f'{population:10,}')
        total_population += population
        
    
    print("\n-------------------------------------------------------------------------------")
    print("  Total Population: ", f'{total_population:10,}')
    print("-------------------------------------------------------------------------------")

In [15]:
def my_station_get_zips_alternative(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    total_population = 0
    
    zip_list = []
    
    for row in rows:
        zip = row[0]
        zip_list.append(zip)
        population = row[1]
        total_population += population
    
    return {str(station): zip_list}

In [16]:
def my_station_get_population(station, miles):
    "given a station, get population with x amount of miles"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    total_population = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        total_population += population
        
    
    return {str(station): int(total_population)}

In [17]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [18]:
cursor = connection.cursor()

### 2.1 Prepping Data: Drop tables if exisits

In [19]:
connection.rollback()

query = """

drop table if exists stations;

"""

cursor.execute(query)

connection.commit()

In [20]:
connection.rollback()

query = """

drop table if exists lines;

"""

cursor.execute(query)

connection.commit()


In [21]:
connection.rollback()

query = """

drop table if exists travel_times;

"""

cursor.execute(query)

connection.commit()


In [22]:
connection.rollback()

query = """

drop table if exists peak_customers

"""

cursor.execute(query)

connection.commit()


### 2.2 Prepping Data: Create tables

In [23]:
connection.rollback()

query = """

create table stations (
  station varchar(32),
  latitude numeric(9,6),
  longitude numeric(9,6),
  transfer_time numeric(3),
  primary key (station));

"""

cursor.execute(query)

connection.commit()

In [24]:
connection.rollback()

query = """

create table lines (
    line varchar(6),
    sequence numeric(2),
    station varchar(32),
    primary key (line, sequence));
"""

cursor.execute(query)

connection.commit()

In [25]:
connection.rollback()

query = """

create table travel_times (
  station_1 varchar(32),
  station_2 varchar(32),
  travel_time numeric(3),
  primary key (station_1, station_2)
);
"""

cursor.execute(query)

connection.commit()

In [26]:
connection.rollback()

query = """

create table peak_customers (
  customer_id varchar(100),
  first_name varchar(100),
  last_name varchar(100),
  street varchar(100),
  city varchar(100),
  state varchar(100),
  zip varchar(100),
  closest_store_id varchar(100),
  distance varchar(100)
  
)
"""

cursor.execute(query)

connection.commit()

### 2.3 Prepping Data: Load data into database from csv files

In [27]:
connection.rollback()

query = """

COPY stations
from '/user/projects/project-3-Tyler-Gustafson/exercise/stations.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [28]:
connection.rollback()

query = """

copy lines
from '/user/projects/project-3-Tyler-Gustafson/exercise/lines.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [29]:
connection.rollback()

query = """

copy travel_times
from '/user/projects/project-3-Tyler-Gustafson/exercise/travel_times.csv' delimiter ',' NULL '' csv header;
"""

cursor.execute(query)

connection.commit()

In [30]:
connection.rollback()

query = """

copy peak_customers
from '/user/projects/project-3-Tyler-Gustafson/code/berkeley_customers_data.csv' delimiter ',' NULL '' csv header;
"""

cursor.execute(query)

connection.commit()

## 3.1 SQL Analysis - customers and routes

In [31]:
# Query the list of stations
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT *
FROM peak_customers
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,customer_id,first_name,last_name,street,city,state,zip,closest_store_id,distance
0,3449,Whitby,Mennithorp,3158 2nd Point,Alameda,CA,94501,1,6
1,3617,Justino,Tenaunt,7635 Caliangt Terrace,Alameda,CA,94501,1,6
2,3616,Chev,Devers,22857 Golf View Terrace,Alameda,CA,94501,1,6
3,3615,Ki,Scrange,326 Utah Junction,Alameda,CA,94501,1,6
4,3614,Letizia,Munkley,2 Calypso Way,Alameda,CA,94501,1,6
...,...,...,...,...,...,...,...,...,...
8133,8095,Emmet,Phippin,78379 Fieldstone Trail,Woodacre,CA,94973,1,23
8134,8096,Ulric,Ravenscroftt,36849 Eagan Way,Woodacre,CA,94973,1,23
8135,8097,Jeremy,Diable,942 Rigney Circle,Woodacre,CA,94973,1,23
8136,8098,Luisa,Symcoxe,291 Thompson Street,Woodacre,CA,94973,1,23


In [32]:
# Query the list of stations
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT station
FROM stations
ORDER BY station
"""

stations = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)['station'].tolist()

In [33]:
all_stations_zip_dictionary = {}

for station in stations:
    result = my_station_get_zips_alternative(station, 7)
    all_stations_zip_dictionary.update(result)
    
all_stations_zip_dictionary

{'12th Street': ['94105',
  '94107',
  '94111',
  '94124',
  '94130',
  '94158',
  '94501',
  '94502',
  '94516',
  '94563',
  '94577',
  '94601',
  '94602',
  '94603',
  '94605',
  '94606',
  '94607',
  '94608',
  '94609',
  '94610',
  '94611',
  '94612',
  '94613',
  '94618',
  '94621',
  '94702',
  '94703',
  '94704',
  '94705',
  '94706',
  '94707',
  '94708',
  '94709',
  '94710',
  '94720'],
 '16th Street Mission': ['94005',
  '94014',
  '94015',
  '94102',
  '94103',
  '94104',
  '94105',
  '94107',
  '94108',
  '94109',
  '94110',
  '94111',
  '94112',
  '94114',
  '94115',
  '94116',
  '94117',
  '94118',
  '94121',
  '94122',
  '94123',
  '94124',
  '94127',
  '94129',
  '94130',
  '94131',
  '94132',
  '94133',
  '94134',
  '94158',
  '94607',
  '94965'],
 '19th Street': ['94105',
  '94107',
  '94124',
  '94130',
  '94158',
  '94501',
  '94502',
  '94516',
  '94563',
  '94577',
  '94601',
  '94602',
  '94603',
  '94605',
  '94606',
  '94607',
  '94608',
  '94609',
  '94610',

In [34]:
# Assuming all_stations_zip_dictionary is your dictionary of stations and associated zip codes
data_for_df = []

for station, zip_codes in all_stations_zip_dictionary.items():
    # Convert the list of zip codes to a string format suitable for SQL IN clause
    zip_codes_str = "'" + "', '".join(zip_codes) + "'"
    
    # Construct SQL query
    query = f"""
    SELECT COUNT(*) as customer_count
    FROM peak_customers
    WHERE zip IN ({zip_codes_str})
    """
    
    # Execute the query and get the result in a DataFrame
    result_df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
    
    # Assuming the query returns a single row with a single column 'customer_count'
    if not result_df.empty:
        customer_count = result_df.iloc[0]['customer_count']
    else:
        customer_count = 0 
    
    # Append the station name and customer count to the data list
    data_for_df.append({'Station': station, 'Current Customer Count Within 7 Miles': customer_count})

# Create a DataFrame from the accumulated data
df_stations_customers = pd.DataFrame(data_for_df)

df_stations_customers['Percentage (%)'] = (df_stations_customers['Current Customer Count Within 7 Miles'] / 8138 * 100).round(2)

# Display the DataFrame
df_stations_customers.sort_values(by='Current Customer Count Within 7 Miles', ascending = False)


,Station,Current Customer Count Within 7 Miles,Percentage (%)
49,West Oakland,4594,56.45
30,North Berkeley,4589,56.39
26,MacArthur,4572,56.18
15,Downtown Berkeley,4554,55.96
5,Ashby,4458,54.78
39,Rockridge,4277,52.56
0,12th Street,4240,52.10
2,19th Street,4204,51.66
33,Orinda,4202,51.63
18,El Cerrito Plaza,4067,49.98


In [35]:
# Query the lines served by the West Oakland Station
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT station,
    line
FROM lines
WHERE station = 'West Oakland'
ORDER BY station, line
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,line
0,West Oakland,blue
1,West Oakland,green
2,West Oakland,red
3,West Oakland,yellow


In [36]:
# Query of the list of stations and the lines they serve
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT station,
    line
FROM lines
ORDER BY station, line
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,line
0,12th Street,orange
1,12th Street,red
2,12th Street,yellow
3,16th Street Mission,blue
4,16th Street Mission,green
...,...,...
109,West Dublin,blue
110,West Oakland,blue
111,West Oakland,green
112,West Oakland,red


In [37]:
# Query the list of all possible line transfers at the West Oakland station and the transfer times
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT a.station, a.line as from_line, b.line as to_line, s.transfer_time
FROM lines a
    JOIN lines b
        ON a.station = b.station AND a.line <> b.line
    JOIN stations s
        ON a.station = s.station
WHERE a.station = 'West Oakland'
ORDER BY a.station, from_line, to_line
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,from_line,to_line,transfer_time
0,West Oakland,blue,green,283
1,West Oakland,blue,red,283
2,West Oakland,blue,yellow,283
3,West Oakland,green,blue,283
4,West Oakland,green,red,283
5,West Oakland,green,yellow,283
6,West Oakland,red,blue,283
7,West Oakland,red,green,283
8,West Oakland,red,yellow,283
9,West Oakland,yellow,blue,283


In [38]:
# Query the list of all possible line transfers and the transfer times
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT a.station, a.line AS from_line, b.line AS to_line, s.transfer_time
FROM lines a
    JOIN lines b 
        ON a.station = b.station AND a.line <> b.line
    JOIN stations s
        ON a.station = s.station
ORDER BY a.station, a.line, b.line

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,from_line,to_line,transfer_time
0,12th Street,orange,red,282
1,12th Street,orange,yellow,282
2,12th Street,red,orange,282
3,12th Street,red,yellow,282
4,12th Street,yellow,orange,282
...,...,...,...,...
203,West Oakland,red,green,283
204,West Oakland,red,yellow,283
205,West Oakland,yellow,blue,283
206,West Oakland,yellow,green,283


In [39]:
# Query the list of all segments between each station and its adjoining stations
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT a.line, a.station AS from_station, b.station AS to_station, t.travel_time
FROM lines a
  JOIN lines b
    ON a.line = b.line AND b.sequence = (a.sequence + 1)
  JOIN travel_times t
    ON (a.station = t.station_1 AND b.station = t.station_2)
        OR (a.station = t.station_2 AND b.station = t.station_1)
ORDER BY line, from_station, to_station
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,line,from_station,to_station,travel_time
0,blue,16th Street Mission,24th Street Mission,120
1,blue,24th Street Mission,Glen Park,180
2,blue,Balboa Park,Daly City,240
3,blue,Bay Fair,San Leandro,240
4,blue,Castro Valley,Bay Fair,240
...,...,...,...,...
103,yellow,Rockridge,MacArthur,240
104,yellow,San Bruno,SFO,240
105,yellow,South San Francisco,San Bruno,240
106,yellow,Walnut Creek,Lafayette,300


## 4.1 Neo4j: Setting up the NoSql graph database

In [76]:
my_neo4j_wipe_out_database()

In [77]:
# Verify the number of nodes and relationships
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


### Only build these out if looking to use shortest path algorithm

In [67]:
# # Query the list of stations and create the departure and arrival nodes in the graph

# connection.rollback()

# query = """

# select station
# from stations
# order by station

# """

# cursor.execute(query)

# connection.rollback()

# rows = cursor.fetchall()

# for row in rows:
    
#     station = row[0]
    
#     my_neo4j_create_node('depart ' + station)
#     my_neo4j_create_node('arrive ' + station)
    


In [78]:
# Verify the number of nodes and relationships
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


In [79]:
# Query the list of stations and the lines they serve, create line nodes, and create relationships between the line nodes and the departure and arrival nodes with weight 0

connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

#Creates a list of tuples
rows = cursor.fetchall()

#iterates over each tuple
for row in rows:
    
    station = row[0]
    line = row[1]
    
    depart = 'depart ' + station
    arrive = 'arrive ' + station
    line_station = line + ' ' + station
    
    my_neo4j_create_node(line_station)
    my_neo4j_create_relationship_one_way(depart, line_station, 0)
    my_neo4j_create_relationship_one_way(line_station, arrive, 0)
    

In [80]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 114
  Relationships: 0
-------------------------


In [81]:
# Query the list of all possible line transfers and the transfer times, create a relationship for each transfer with the transfer time as the weight
connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    from_station = from_line + ' ' + station
    to_station = to_line + ' ' + station
    
    my_neo4j_create_relationship_one_way(from_station, to_station, transfer_time)
    

In [82]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 114
  Relationships: 208
-------------------------


In [83]:
# Query the list of all segments between each station and its adjoining stations, create a relationship for each segment both ways
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = line + ' ' + row[1]
    to_station = line + ' ' + row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)
    

In [84]:
# Verify the number of nodes and relationships
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 114
  Relationships: 424
-------------------------


## 4.2 Neo4j: Applying NoSql Graph Algorithms

#### Algorithm | Shortest Path (make sure depart / arrive are in the model)

In [75]:
# Shortest path between Embarcadero and Civic Center
my_neo4j_shortest_path('depart Embarcadero', 'arrive Civic Center')


--------------------------------
   Total Cost (Seconds):  240
   Minutes:  4.0
--------------------------------
depart Embarcadero, 0, 0
yellow Embarcadero, 0, 0
yellow Montgomery Street, 60, 60
yellow Powell Street, 120, 180
yellow Civic Center, 60, 240
arrive Civic Center, 0, 240


#### Algorithm | Harmonic Centrality (make sure to EXCLUDE depart / arrive in the model)
 

In [51]:
# Applying Harmonic Centrality
pd.set_option('display.max_rows', None)


query = """

CALL gds.alpha.closeness.harmonic.stream('ds_graph', {})
YIELD nodeId, centrality
RETURN gds.util.asNode(nodeId).name AS name, centrality as closeness
ORDER BY centrality DESC

"""

my_neo4j_run_query_pandas(query)

,name,closeness
0,green West Oakland,0.226464
1,blue West Oakland,0.226162
2,yellow West Oakland,0.225898
3,red West Oakland,0.225645
4,green Embarcadero,0.219591
5,blue Embarcadero,0.219339
6,yellow Embarcadero,0.219332
7,red Embarcadero,0.219117
8,green Lake Merritt,0.215391
9,blue Lake Merritt,0.215022


#### Algorithm | Louvain Modularity - Groupings (make sure to EXCLUDE depart / arrive in the model)

In [52]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 'LINK', 
                      {relationshipProperties: 'track_miles'})
"""

session.run(query)

In [53]:
query = """

CALL gds.louvain.stream('ds_graph', {includeIntermediateCommunities: true})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
ORDER BY community, name ASC

"""

station_groupings_df = my_neo4j_run_query_pandas(query)


In [54]:
station_groupings_df['community'].value_counts()

103    20
19     16
78     16
96     14
107    13
97     10
90      9
29      8
33      8
Name: community, dtype: int64

In [55]:
station_groupings_df['Station'] = station_groupings_df['name'].apply(lambda x: ' '.join(x.split(' ')[1:]))
station_groupings_df

,name,community,intermediate_community,Station
0,blue 16th Street Mission,19,"[2, 19, 19]",16th Street Mission
1,blue 24th Street Mission,19,"[2, 19, 19]",24th Street Mission
2,blue Civic Center,19,"[19, 19, 19]",Civic Center
3,blue Glen Park,19,"[2, 19, 19]",Glen Park
4,green 16th Street Mission,19,"[2, 19, 19]",16th Street Mission
5,green 24th Street Mission,19,"[2, 19, 19]",24th Street Mission
6,green Civic Center,19,"[19, 19, 19]",Civic Center
7,green Glen Park,19,"[2, 19, 19]",Glen Park
8,red 16th Street Mission,19,"[2, 19, 19]",16th Street Mission
9,red 24th Street Mission,19,"[2, 19, 19]",24th Street Mission


#### Algorithm | Geodesic Fencing - within 2 miles (make sure to EXCLUDE depart / arrive in the model)


In [56]:
# Zip codes with population within 2 miles of the coliseum station
my_station_get_zips('Coliseum',2)


-------------------------------------------------------------------------------
  Zip Codes within 2 mile(s) of Coliseum BART Station
-------------------------------------------------------------------------------

     zip: 94601   population:      52,299
     zip: 94603   population:      34,593
     zip: 94613   population:         861
     zip: 94621   population:      35,287

-------------------------------------------------------------------------------
  Total Population:     123,040
-------------------------------------------------------------------------------


In [57]:
# ALl zip codes and population with 1,2,3,4,5 miles of downtown berkeley station
for x in range(1,6):
    my_station_get_zips('Downtown Berkeley', x)


-------------------------------------------------------------------------------
  Zip Codes within 1 mile(s) of Downtown Berkeley BART Station
-------------------------------------------------------------------------------

     zip: 94702   population:      17,092
     zip: 94703   population:      21,937
     zip: 94704   population:      29,190
     zip: 94709   population:      11,740
     zip: 94720   population:       2,971

-------------------------------------------------------------------------------
  Total Population:      82,930
-------------------------------------------------------------------------------

-------------------------------------------------------------------------------
  Zip Codes within 2 mile(s) of Downtown Berkeley BART Station
-------------------------------------------------------------------------------

     zip: 94618   population:      17,041
     zip: 94702   population:      17,092
     zip: 94703   population:      21,937
     zip: 94704   pop

In [58]:
all_stations_population_dictionary = {}

for station in stations:
    result = my_station_get_population(station, 2)
    all_stations_population_dictionary.update(result)
    
population_df = pd.DataFrame.from_dict(all_stations_population_dictionary, orient='index').reset_index()
population_df.columns = ['Station', 'Population']

In [62]:
joined_df = pd.merge(population_df,station_groupings_df, on='Station',how='inner')
joined_df.sort_values('Population',ascending = False)

,Station,Population,name,community,intermediate_community
30,Civic Center,347795,yellow Civic Center,19,"[19, 19, 19]"
29,Civic Center,347795,red Civic Center,19,"[19, 19, 19]"
28,Civic Center,347795,green Civic Center,19,"[19, 19, 19]"
27,Civic Center,347795,blue Civic Center,19,"[19, 19, 19]"
3,16th Street Mission,339093,blue 16th Street Mission,19,"[2, 19, 19]"
4,16th Street Mission,339093,green 16th Street Mission,19,"[2, 19, 19]"
5,16th Street Mission,339093,red 16th Street Mission,19,"[2, 19, 19]"
6,16th Street Mission,339093,yellow 16th Street Mission,19,"[2, 19, 19]"
10,24th Street Mission,315201,blue 24th Street Mission,19,"[2, 19, 19]"
11,24th Street Mission,315201,green 24th Street Mission,19,"[2, 19, 19]"


In [63]:
df_no_duplicates_subset = joined_df.drop_duplicates(subset='Station')
df_no_duplicates_subset = df_no_duplicates_subset.drop(columns=['intermediate_community', 'name'])

In [64]:
top_3_per_community = df_no_duplicates_subset.groupby('community').head(3)
top_3_per_community.sort_values(by=['community','Population'],ascending = [True, False])

,Station,Population,community
27,Civic Center,347795,19
3,16th Street Mission,339093,19
10,24th Street Mission,315201,19
17,Balboa Park,253123,29
38,Daly City,231056,29
78,North Berkeley,134802,33
45,El Cerrito del Norte,81472,33
47,El Cerrito Plaza,70450,33
86,Powell Street,303556,78
74,Montgomery Street,268802,78


In [66]:
filtered_df = top_3_per_community[top_3_per_community['community'] == 19]
filtered_df.sort_values('Population',ascending = False)

,Station,Population,community
27,Civic Center,347795,19
3,16th Street Mission,339093,19
10,24th Street Mission,315201,19
